In [16]:
import sys
sys.path.append("/Users/haraldmack/Development/iSparrow")
from datetime import datetime
import time
import logging
from watchdog.observers import Observer
from watchdog.events import LoggingEventHandler, FileSystemEventHandler
from pathlib import Path
import tests.set_up_sparrow_env as sp
import iSparrow.utils as utils
from iSparrow import SparrowRecording
from iSparrow import ModelBase
from iSparrow import PreprocessorBase
import pandas as pd


In [2]:
# make a mock install of sparrow. will be invisible in the future
sp.install()

Creating iSparrow folders and downloading data... 
...reading config from /Users/haraldmack/Development/iSparrow/config/install_cfg.yml
...Making direcotries...
/Users/haraldmack/iSparrow /Users/haraldmack/iSparrow/models /Users/haraldmack/iSparrow_data /Users/haraldmack/iSparrow_output /Users/haraldmack/iSparrow/example
... Downloading model files...
... Downloading example files...
Installation finished


In [3]:
class AnalysisEventHandler(FileSystemEventHandler):
    def __init__(self, callback: callable):
        self.callback = callback

    def on_created(self, event):
        print(event.event_type, event.src_path)
        if Path(event.src_path).is_file() and Path(event.src_path).suffix == ".wav":
            self.callback(event.src_path)

In [30]:
# FIXME: add species predictor
class Runner:

    def _load_model(self):
        self.preprocessor_module = utils.load_module(
            "ppm", self.model_dir / Path(self.model_name) / "preprocessor.py"
        )

        self.model_module = utils.load_module(
            "md", self.model_dir / Path(self.model_name) / "model.py"
        )

    def __init__(
        self,
        indir: str,
        outdir: str,
        model_dir: str,
        model_name: str,
        preprocessor_config: dict = {},
        model_config: dict = {},
        recording_config: dict = {},
    ):
        self.input = Path(indir)
        self.output = Path(outdir)
        self.model_dir = Path(model_dir)
        self.model_name = model_name
        self.proprocessor_module = None
        self.model_module = None
        self.model = None
        self.preprocessor = None

        self._load_model()

        self.preprocessor = self.preprocessor_module.Preprocessor(**preprocessor_config)

        self.model = self.model_module.Model(
            self.model_dir / model_name, **model_config
        )

        self.recording = SparrowRecording(
            self.preprocessor, self.model, "", **recording_config
        )

        self.results = []

        self.start_time = None
        self.end_time = None

    def change_model(
        self,
        model_name,
        preprocessor_config: dict = {},
        model_config: dict = {},
    ):

        raise RuntimeError("Not yet supported, only untested skeleton code exists")

        self.model_name = model_name

        self._load_model()

        self.preprocessor = self.preprocessor_module.Preprocessor(**preprocessor_config)

        self.model = self.model_module.Model(
            self.model_dir / model_name, **model_config
        )

        self.recording.set_analyzer(model, preprocessor)

    def analyze(self, filename: str):
        self.recording.path = filename

        self.recording.analyze()

        self.results.extend(self.recording.detections)

        if len(self.results) >= 500:
            self.save_results()
            self.start_time = datetime.now().strftime("%y%m%d_%H%M%S")

    def save_results(self):
        self.end_time = datetime.now().strftime("%y%m%d_%H%M%S")

        pd.DataFrame(self.results).to_csv(
            self.output / Path(f"{self.start_time}_{self.end_time}.csv")
        )

    def run(self):
        observer = Observer()
        event_handler = AnalysisEventHandler(self.analyze)
        observer.schedule(event_handler, self.input, recursive=True)
        self.start_time = datetime.now().strftime("%y%m%d_%H%M%S")
        observer.start()

        try:
            while True:
                time.sleep(1)
        except KeyboardInterrupt:
            observer.stop()
        except Exception:
            print("Something went wrong")
            observer.stop()
        observer.join()

In [31]:
preprocessor_cfg = {
    "sample_rate": 48000,
    "overlap": 0.0,
    "sample_secs": 3.0,
    "resample_type": "kaiser_fast",
}

model_cfg = {
    "num_threads": 1,
    "sigmoid_sensitivity": 1.0,
    "species_list_file": None,
}

recording_cfg = {
    "date": None,
    "lat": None,
    "lon": None,
    "species_presence_threshold": 0.03,
    "min_conf": 0.25,
    "species_predictor": None,
}

In [33]:
runner = Runner(
    Path.home() / "iSparrow_data",
    Path.home() / "iSparrow_output",
    Path.home() / "iSparrow/models",
    "birdnet_default",
    preprocessor_cfg,
    model_cfg,
    recording_cfg,
)

Labels loaded.


In [34]:
runner.run()

created /Users/haraldmack/iSparrow_data/240409_101212.wav
read audio data of duration 24.0 from /Users/haraldmack/iSparrow_data/240409_101212.wav with sampling rate 48000
process audio data custom
process audio data custom: complete, read  8 chunks.
name of model:  birdnet_default
created /Users/haraldmack/iSparrow_data/240409_101236.wav
read audio data of duration 24.0 from /Users/haraldmack/iSparrow_data/240409_101236.wav with sampling rate 48000
process audio data custom
process audio data custom: complete, read  8 chunks.
name of model:  birdnet_default


In [35]:
runner.results

[{'start': 9.0,
  'end': 12.0,
  'label': 'Turdus merula_Eurasian Blackbird',
  'confidence': 0.3805372},
 {'start': 12.0,
  'end': 15.0,
  'label': 'Turdus merula_Eurasian Blackbird',
  'confidence': 0.3738132},
 {'start': 15.0,
  'end': 18.0,
  'label': 'Turdus hortulorum_Gray-backed Thrush',
  'confidence': 0.52868617},
 {'start': 18.0,
  'end': 21.0,
  'label': 'Turdus merula_Eurasian Blackbird',
  'confidence': 0.9195597},
 {'start': 21.0,
  'end': 24.0,
  'label': 'Turdus merula_Eurasian Blackbird',
  'confidence': 0.44440657},
 {'start': 21.0,
  'end': 24.0,
  'label': 'Sylvia atricapilla_Eurasian Blackcap',
  'confidence': 0.25587997},
 {'start': 12.0,
  'end': 15.0,
  'label': 'Turdus merula_Eurasian Blackbird',
  'confidence': 0.70951295}]

In [36]:
runner.save_results()

In [28]:
res = [{'start': 9.0,
   'end': 12.0,
   'label': 'Turdus merula_Eurasian Blackbird',
   'confidence': 0.3805372},
  {'start': 12.0,
   'end': 15.0,
   'label': 'Turdus merula_Eurasian Blackbird',
   'confidence': 0.3738132},
  {'start': 15.0,
   'end': 18.0,
   'label': 'Turdus hortulorum_Gray-backed Thrush',
   'confidence': 0.52868617},
  {'start': 18.0,
   'end': 21.0,
   'label': 'Turdus merula_Eurasian Blackbird',
   'confidence': 0.9195597},]

In [29]:
pd.DataFrame(res)

,start,end,label,confidence
0,9.0,12.0,Turdus merula_Eurasian Blackbird,0.380537
1,12.0,15.0,Turdus merula_Eurasian Blackbird,0.373813
2,15.0,18.0,Turdus hortulorum_Gray-backed Thrush,0.528686
3,18.0,21.0,Turdus merula_Eurasian Blackbird,0.919560
